# APP DATA

In [ ]:
# @title
from google.colab import drive
drive.mount('/content/drive')

# Módulos base:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 200
import os
import numpy as np
import math
from collections import Counter
import pickle
import warnings
warnings.filterwarnings("ignore")

# Tamaño de las celdas del Notebook:
from IPython.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
# Directorio de trabajo:
os.chdir('/content/drive/MyDrive/PERSONAL AGJM/PROYECTOS ML Y ESTAD./SCORE CARD/Home Credit Default Risk')

Mounted at /content/drive


In [ ]:
# @title
# Función completitud de columnas:
def completitud(df):
    """Función que calcula el número de registros
    nulos por columna en un DataFrame.

    Parameters:
    ----------
    df : DataFrame

    Returns
    -------
    Dataframe:
    """
    comp=pd.DataFrame(df.isnull().sum())
    comp.reset_index(inplace=True)
    comp=comp.rename(columns={"index":"Columna",0:"Total_reg_nulos"})
    comp["Per_completitud"]=(1-comp["Total_reg_nulos"]/df.shape[0])*100
    comp=comp.sort_values(by="Per_completitud",ascending=True)
    #comp.reset_index(drop=True,inplace=True)
    return comp

# APP DATA

In [ ]:
app_data=pd.read_csv("Data/application_train.csv")

In [ ]:
app_data.sample(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
305381,453812,0,Cash loans,M,N,Y,1,117000.0,425326.5,22266.0,425326.5,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.008474,-9029,-1590,-1576.0,-1570,NaN,1,1,0,1,0,0,Laborers,3.0,2,2,WEDNESDAY,10,0,0,0,0,1,1,Construction,0.075449,0.576886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-563.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
213261,347131,0,Cash loans,F,N,Y,1,157500.0,1223010.0,51817.5,1125000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.009549,-11281,-3767,-1955.0,-2582,NaN,1,1,0,1,0,0,Laborers,3.0,2,2,THURSDAY,17,0,0,0,0,0,0,Self-employed,NaN,0.668350,0.771362,0.1485,NaN,0.9811,NaN,NaN,NaN,0.0345,0.1667,NaN,0.0663,NaN,0.0741,NaN,0.0158,0.1513,NaN,0.9811,NaN,NaN,NaN,0.0345,0.1667,NaN,0.0678,NaN,0.0772,NaN,0.0167,0.1499,NaN,0.9811,NaN,NaN,NaN,0.0345,0.1667,NaN,0.0674,NaN,0.0754,NaN,0.0161,NaN,block of flats,0.0617,"Stone, brick",No,1.0,1.0,1.0,1.0,-355.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0
35077,140635,0,Cash loans,M,Y,Y,0,112500.0,675000.0,21906.0,675000.0,Unaccompanied,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,0.025164,-21065,365243,-7582.0,-4198,14.0,1,0,0,1,0,0,NaN,2.0,2,2,WEDNESDAY,15,0,0,0,0,0,0,XNA,NaN,0.328747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-2637.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


# PREVIOUS APPLICATION

In [ ]:
prev_app=pd.read_csv("Data/previous_application.csv")
pos_cash_data=pd.read_csv("Data/POS_CASH_balance.csv")
ip_data=pd.read_csv("Data/installments_payments.csv")
cc_data=pd.read_csv("Data/credit_card_balance.csv")

In [ ]:
app_data.shape[1]+prev_app.shape[1]+pos_cash_data.shape[1]+ip_data.shape[1]+cc_data.shape[1]

198

In [ ]:
app_data.shape[0]+prev_app.shape[0]+pos_cash_data.shape[0]+ip_data.shape[0]+cc_data.shape[0]

29424796

In [ ]:
vars_pa=["SK_ID_PREV","SK_ID_CURR","AMT_CREDIT","AMT_APPLICATION",
         "NAME_CONTRACT_STATUS","DAYS_DECISION","NAME_PORTFOLIO",
         "NAME_PRODUCT_TYPE"]
vars_pc=["SK_ID_PREV","SK_ID_CURR","SK_DPD"]
vars_ip=["SK_ID_PREV","SK_ID_CURR","DAYS_INSTALMENT","DAYS_ENTRY_PAYMENT","AMT_INSTALMENT","AMT_PAYMENT"]
vars_cc=["SK_ID_PREV","SK_ID_CURR","AMT_BALANCE","AMT_CREDIT_LIMIT_ACTUAL","AMT_DRAWINGS_ATM_CURRENT",
         "AMT_DRAWINGS_CURRENT","CNT_DRAWINGS_ATM_CURRENT","CNT_DRAWINGS_CURRENT","CNT_INSTALMENT_MATURE_CUM",
         "NAME_CONTRACT_STATUS","SK_DPD"]
prev_app=prev_app[vars_pa]
pos_cash_data=pos_cash_data[vars_pc]
ip_data=ip_data[vars_ip]
cc_data=cc_data[vars_cc]

#PREV_APP


In [ ]:
#CONTRACT STATUS:
prev_app["APP_APPROVED"]=prev_app["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x=="Approved" else 0)
prev_app["APP_REFUSED"]=prev_app["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x=="Refused" else 0)
prev_app["APP_CANCELED"]=prev_app["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x=="Canceled" else 0)
prev_app["APP_UNSED_OFFER"]=prev_app["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x=="Unused offer" else 0)
#NAME PORTAFOLIO:
prev_app["POS_PORT"]=prev_app["NAME_PORTFOLIO"].apply(lambda x: 1 if x=="POS" else 0)
prev_app["CASH_PORT"]=prev_app["NAME_PORTFOLIO"].apply(lambda x: 1 if x=="Cash" else 0)
prev_app["CARD_PORT"]=prev_app["NAME_PORTFOLIO"].apply(lambda x: 1 if x=="Cards" else 0)
prev_app["CAR_PORT"]=prev_app["NAME_PORTFOLIO"].apply(lambda x: 1 if x=="Cars" else 0)
prev_app["OTHER_PORT"]=prev_app["NAME_PORTFOLIO"].apply(lambda x: 1 if x=="XNA" else 0)
#X SELL:
prev_app["X_SELL"]=prev_app["NAME_PRODUCT_TYPE"].apply(lambda x: 1 if x=='x-sell' else 0)
#RATIO AMOUNT APPLICATION:
prev_app["RATIO_AMT_APP"]=prev_app["AMT_CREDIT"]/prev_app["AMT_APPLICATION"]
prev_app["RATIO_AMT_APP"]=prev_app["RATIO_AMT_APP"].fillna(0)

In [ ]:
#CREAMOS COLUMNAS A PARTIR DE AGRUPACIONES
agg_prev_app=prev_app.groupby("SK_ID_CURR").agg({"RATIO_AMT_APP":['max','min'],"DAYS_DECISION":["max","min","mean"],
                                    "APP_APPROVED":'sum',"APP_REFUSED":'sum',"APP_CANCELED":'sum',
                                    "APP_UNSED_OFFER":'sum',"POS_PORT":'sum',"CASH_PORT":'sum',
                                    "CARD_PORT":'sum','CAR_PORT':'sum',"OTHER_PORT":'sum',"X_SELL":'sum'})

In [ ]:
agg_prev_app.sample(3)

RATIO_AMT_APP           DAYS_DECISION                     \
                     max       min           max   min         mean   
SK_ID_CURR                                                            
300825          1.251820  1.000000          -375 -2777 -1107.500000   
411292          1.216467  0.000000           -12 -1879  -606.714286   
115550          1.000000  0.795862          -221 -1833 -1027.000000   

           APP_APPROVED APP_REFUSED APP_CANCELED APP_UNSED_OFFER POS_PORT  \
                    sum         sum          sum             sum      sum   
SK_ID_CURR                                                                  
300825                4           0            0               0        1   
411292                7           1            6               0        2   
115550                2           0            0               0        2   

           CASH_PORT CARD_PORT CAR_PORT OTHER_PORT X_SELL  
                 sum       sum      sum        sum    sum  
SK_ID_CURR                                                 
300825             3         0        0          0      3  
411292             5         1        0          6      5  
115550             0         0        0          0      0

In [ ]:
#CREMAOS ÍNDICES EN LUGAR DE DEJAR MULTI ÍNDICES
agg_prev_app.columns=pd.Index(["pre_app"+"_"+c[0]+"_"+c[1] for c in  agg_prev_app.columns.to_list()])

In [ ]:
agg_prev_app["pre_app_RATIO_AMT_APP_max"]=agg_prev_app["pre_app_RATIO_AMT_APP_max"].apply(str_to_int)
agg_prev_app["pre_app_RATIO_AMT_APP_min"]=agg_prev_app["pre_app_RATIO_AMT_APP_min"].apply(str_to_int)
agg_prev_app=agg_prev_app.reset_index()

In [ ]:
print(agg_prev_app.shape)
agg_prev_app.sample(3)

(338857, 16)


,SK_ID_CURR,pre_app_RATIO_AMT_APP_max,pre_app_RATIO_AMT_APP_min,pre_app_DAYS_DECISION_max,pre_app_DAYS_DECISION_min,pre_app_DAYS_DECISION_mean,pre_app_APP_APPROVED_sum,pre_app_APP_REFUSED_sum,pre_app_APP_CANCELED_sum,pre_app_APP_UNSED_OFFER_sum,pre_app_POS_PORT_sum,pre_app_CASH_PORT_sum,pre_app_CARD_PORT_sum,pre_app_CAR_PORT_sum,pre_app_OTHER_PORT_sum,pre_app_X_SELL_sum
170075,278905,1.088000,1.088000,-179,-179,-179.0,1,0,0,0,1,0,0,0,0,0
133805,240673,1.205756,0.955378,-590,-1483,-932.6,5,0,0,0,3,1,1,0,0,2
93886,198691,1.000000,0.988984,-1981,-2563,-2369.0,2,1,0,0,3,0,0,0,0,0


# POS CASH DATA

In [ ]:
pos_cash_data.sample(3)

,SK_ID_PREV,SK_ID_CURR,SK_DPD
7913258,1234303,221079,0
4093186,2049643,409014,0
7231126,1040128,254636,0


In [ ]:
agg_pos_cash_data=pos_cash_data.groupby("SK_ID_CURR").agg({"SK_DPD":["max","mean"]})

In [ ]:
#CREMAOS ÍNDICES EN LUGAR DE DEJAR MULTI ÍNDICES
agg_pos_cash_data.columns=pd.Index(["pos_cash"+"_"+c[0]+"_"+c[1] for c in  agg_pos_cash_data.columns.to_list()])
agg_pos_cash_data.reset_index(inplace=True)

In [ ]:
print(agg_pos_cash_data.shape)
agg_pos_cash_data.sample(3)

(337252, 3)


,SK_ID_CURR,pos_cash_SK_DPD_max,pos_cash_SK_DPD_mean
148031,256503,0,0.0
234750,348101,0,0.0
272362,387395,0,0.0


# CREDIT CARD_BALANCE

In [ ]:
cc_data.sample(3)

,SK_ID_PREV,SK_ID_CURR,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD
1015685,1104865,386268,0.000,0,0.0,0.0,0.0,0,19.0,Active,0
1084628,2818795,373575,162895.095,180000,0.0,0.0,0.0,0,4.0,Active,0
1099261,1590911,198304,1188.405,112500,0.0,0.0,0.0,0,43.0,Active,0


In [ ]:
cc_data["CARDS_ACTIVE"]=cc_data["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x=='Active' else 0)
cc_data["CARDS_COMPLETED"]=cc_data["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x=='Completed' else 0)
cc_data["CARDS_REFUSED"]=cc_data["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x=='Refused' else 0)
cc_data["CARDS_APROVED"]=cc_data["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x=='Approved' else 0)
cc_data["CARDS_DEMAND"]=cc_data["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x=='Demand' else 0)

In [ ]:
agg_cc_data=cc_data.groupby("SK_ID_CURR").agg({'AMT_BALANCE':'mean','AMT_CREDIT_LIMIT_ACTUAL':['max','mean'],
                                   'AMT_DRAWINGS_ATM_CURRENT':'mean','AMT_DRAWINGS_CURRENT':['mean','max'],
                                   'CNT_DRAWINGS_ATM_CURRENT':['mean','max'],'CNT_DRAWINGS_CURRENT':['mean','max'],
                                   'CNT_INSTALMENT_MATURE_CUM':'mean',"CARDS_ACTIVE":'sum',"CARDS_COMPLETED":'sum', #indican el #de meses con la marca
                                   "CARDS_REFUSED":'sum',"CARDS_APROVED":'sum',"CARDS_DEMAND":'sum',
                                   'SK_DPD':['max','mean']
                                   })

In [ ]:
#CREMAOS ÍNDICES EN LUGAR DE DEJAR MULTI ÍNDICES
agg_cc_data.columns=pd.Index(["cc"+"_"+c[0]+"_"+c[1] for c in  agg_cc_data.columns.to_list()])
agg_cc_data.reset_index(inplace=True)

In [ ]:
agg_cc_data.sample(3)

,SK_ID_CURR,cc_AMT_BALANCE_mean,cc_AMT_CREDIT_LIMIT_ACTUAL_max,cc_AMT_CREDIT_LIMIT_ACTUAL_mean,cc_AMT_DRAWINGS_ATM_CURRENT_mean,cc_AMT_DRAWINGS_CURRENT_mean,cc_AMT_DRAWINGS_CURRENT_max,cc_CNT_DRAWINGS_ATM_CURRENT_mean,cc_CNT_DRAWINGS_ATM_CURRENT_max,cc_CNT_DRAWINGS_CURRENT_mean,cc_CNT_DRAWINGS_CURRENT_max,cc_CNT_INSTALMENT_MATURE_CUM_mean,cc_CARDS_ACTIVE_sum,cc_CARDS_COMPLETED_sum,cc_CARDS_REFUSED_sum,cc_CARDS_APROVED_sum,cc_CARDS_DEMAND_sum,cc_SK_DPD_max,cc_SK_DPD_mean
87762,400544,85086.390000,135000,105000.0,13950.0,13950.000000,90000.000,2.833333,7.0,2.833333,7,5.500000,12,0,0,0,0,0,0.0
2393,108389,204170.477727,225000,225000.0,25110.0,22827.272727,193500.000,1.500000,9.0,1.363636,9,4.090909,11,0,0,0,0,0,0.0
55003,289662,18554.586000,135000,72000.0,0.0,9584.415000,32625.225,0.000000,0.0,3.066667,15,6.000000,15,0,0,0,0,0,0.0


# INSTALLMENT **PAYMENTS**

**COLUMNAS CANDIDATAS:**

*   DAYS_INSTALMENT
*   DAYS_ENTRY_PAYMENT
*   AMT_INSTALMENT
*   AMT_PAYMENT



In [ ]:
ip_data.sample(3)

,SK_ID_PREV,SK_ID_CURR,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
10275610,2080974,367995,-1368.0,-1352.0,21144.150,5.400
1270077,2784084,194080,-307.0,-320.0,8090.595,8090.595
7277790,1486830,206061,-587.0,-590.0,24434.100,45000.000


In [ ]:
agg_ip_data=ip_data.groupby("SK_ID_CURR").agg({"DAYS_INSTALMENT":["max","min","mean"],"DAYS_ENTRY_PAYMENT":["max","min","mean"],
                                   "AMT_INSTALMENT":["max","min","mean"],"AMT_PAYMENT":["max","min","mean"]})

In [ ]:
agg_ip_data.columns=pd.Index(["ip"+"_"+c[0]+"_"+c[1] for c in  agg_ip_data.columns.to_list()])
agg_ip_data.reset_index(inplace=True)

In [ ]:
agg_ip_data.sample(3)

,SK_ID_CURR,ip_DAYS_INSTALMENT_max,ip_DAYS_INSTALMENT_min,ip_DAYS_INSTALMENT_mean,ip_DAYS_ENTRY_PAYMENT_max,ip_DAYS_ENTRY_PAYMENT_min,ip_DAYS_ENTRY_PAYMENT_mean,ip_AMT_INSTALMENT_max,ip_AMT_INSTALMENT_min,ip_AMT_INSTALMENT_mean,ip_AMT_PAYMENT_max,ip_AMT_PAYMENT_min,ip_AMT_PAYMENT_mean
96291,200988,-100.0,-430.0,-268.461538,-101.0,-431.0,-270.076923,50584.050,50544.18,50580.983077,50584.050,118.80,46689.902308
154431,262096,-8.0,-2438.0,-1001.379310,-14.0,-2457.0,-1011.045977,63909.000,137.70,5120.383707,63909.000,137.70,4873.838017
311224,426617,-168.0,-479.0,-310.307692,-168.0,-509.0,-330.000000,173449.575,38028.06,55073.700000,173449.575,38028.06,55073.700000


# BUREAU DATA


## BUREAU BALANCE

In [ ]:
bb_data=pd.read_csv("Data/bureau_balance.csv")

In [ ]:
bb_data["0PD"]=bb_data["STATUS"].apply(lambda x: 1 if x=='0' else 0)
bb_data["1_30PD"]=bb_data["STATUS"].apply(lambda x: 1 if x=='1' else 0)
bb_data["31_60PD"]=bb_data["STATUS"].apply(lambda x: 1 if x=='2' else 0)
bb_data["120PD_MORE"]=bb_data["STATUS"].apply(lambda x: 1 if x=='3' else 0)


In [ ]:
agg_bb_data=bb_data.groupby("SK_ID_BUREAU").agg({"MONTHS_BALANCE":["min","max","mean"],"31_60PD":['max','sum'],"0PD":['max','sum'],
                                                 "120PD_MORE":['max','sum']})

In [ ]:
agg_bb_data.columns=pd.Index(["bb"+"_"+c[0]+"_"+c[1] for c in  agg_bb_data.columns.to_list()])
agg_bb_data.reset_index(inplace=True)

In [ ]:
agg_bb_data.sort_values(by='SK_ID_BUREAU').head(5)

,SK_ID_BUREAU,bb_MONTHS_BALANCE_min,bb_MONTHS_BALANCE_max,bb_MONTHS_BALANCE_mean,bb_31_60PD_max,bb_31_60PD_sum,bb_0PD_max,bb_0PD_sum,bb_120PD_MORE_max,bb_120PD_MORE_sum
0,5001709,-96,0,-48.0,0,0,0,0,0,0
1,5001710,-82,0,-41.0,0,0,1,5,0,0
2,5001711,-3,0,-1.5,0,0,1,3,0,0
3,5001712,-18,0,-9.0,0,0,1,10,0,0
4,5001713,-21,0,-10.5,0,0,0,0,0,0


## BUREAU DATA JOIN BUREAU BALANCE

In [ ]:
b_data=pd.read_csv("Data/bureau.csv")

In [ ]:
bureau_data=b_data.merge(agg_bb_data,on='SK_ID_BUREAU',how='left')

## AGRUPACIONES BUREAU DATA

In [ ]:
bureau_data.sample(3)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,bb_MONTHS_BALANCE_min,bb_MONTHS_BALANCE_max,bb_MONTHS_BALANCE_mean,bb_31_60PD_max,bb_31_60PD_sum,bb_0PD_max,bb_0PD_sum,bb_120PD_MORE_max,bb_120PD_MORE_sum
1425072,127842,6738938,Closed,currency 1,-1430,0,-699.0,-700.0,NaN,0,306270.00,0.0,0.0,0.0,Consumer credit,-700,6880.5,-47.0,-42.0,-44.5,0.0,0.0,1.0,5.0,0.0,0.0
976913,169709,6208214,Closed,currency 1,-1573,0,-1026.0,-1105.0,NaN,0,270000.00,0.0,0.0,0.0,Consumer credit,-224,NaN,-51.0,-9.0,-30.0,0.0,0.0,1.0,5.0,0.0,0.0
764630,321933,6234079,Closed,currency 1,-786,0,-393.0,-511.0,NaN,0,225719.64,0.0,NaN,0.0,Consumer credit,-99,NaN,-25.0,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
bureau_data["CREDITS_ACTIVE"]=bureau_data["CREDIT_ACTIVE"].apply(lambda x: 1 if x=='Active' else 0)
bureau_data["CREDITS_CLOSED"]=bureau_data["CREDIT_ACTIVE"].apply(lambda x: 1 if x=='Closed' else 0)
bureau_data["CONSUMER_CREDITS"]=bureau_data["CREDIT_TYPE"].apply(lambda x: 1 if x=='Consumer credit' else 0)
bureau_data["CREDITS_CARDS"]=bureau_data["CREDIT_TYPE"].apply(lambda x: 1 if x=='Credit card' else 0)
bureau_data["CAR_LOANS"]=bureau_data["CREDIT_TYPE"].apply(lambda x: 1 if x=='Car loan' else 0)
bureau_data["MICRO_LOANS"]=bureau_data["CREDIT_TYPE"].apply(lambda x: 1 if x=='Microloan' else 0)

In [ ]:
agg_bureau_data=bureau_data.groupby("SK_ID_CURR").agg({"CREDITS_ACTIVE":'sum',"CREDITS_CLOSED":'sum',"CONSUMER_CREDITS":'sum',
                                       "CREDITS_CARDS":'sum',"CAR_LOANS":'sum',"MICRO_LOANS":'sum'
                                       ,"DAYS_CREDIT":['mean','max','min'],"CREDIT_DAY_OVERDUE":['max','mean'],
                                       "DAYS_CREDIT_ENDDATE":['mean','max','min'],"CNT_CREDIT_PROLONG":['sum','mean'],
                                       "AMT_CREDIT_SUM":["mean","max"],"AMT_CREDIT_SUM_DEBT":["mean","max"],
                                       "AMT_CREDIT_SUM_LIMIT":['sum','mean','max'],"AMT_CREDIT_SUM_OVERDUE":['max','mean'],
                                       "DAYS_CREDIT_UPDATE":["max","min","mean"],
                                       "bb_31_60PD_max":'sum',"bb_0PD_max":'sum', "bb_120PD_MORE_max":'sum'})

In [ ]:
agg_bureau_data.columns=pd.Index(["bd"+"_"+c[0]+"_"+c[1] for c in  agg_bureau_data.columns.to_list()])
agg_bureau_data.reset_index(inplace=True)

In [ ]:
print(agg_bureau_data.shape)
agg_bureau_data.sample(3)

(305811, 32)


,SK_ID_CURR,bd_CREDITS_ACTIVE_sum,bd_CREDITS_CLOSED_sum,bd_CONSUMER_CREDITS_sum,bd_CREDITS_CARDS_sum,bd_CAR_LOANS_sum,bd_MICRO_LOANS_sum,bd_DAYS_CREDIT_mean,bd_DAYS_CREDIT_max,bd_DAYS_CREDIT_min,bd_CREDIT_DAY_OVERDUE_max,bd_CREDIT_DAY_OVERDUE_mean,bd_DAYS_CREDIT_ENDDATE_mean,bd_DAYS_CREDIT_ENDDATE_max,bd_DAYS_CREDIT_ENDDATE_min,bd_CNT_CREDIT_PROLONG_sum,bd_CNT_CREDIT_PROLONG_mean,bd_AMT_CREDIT_SUM_mean,bd_AMT_CREDIT_SUM_max,bd_AMT_CREDIT_SUM_DEBT_mean,bd_AMT_CREDIT_SUM_DEBT_max,bd_AMT_CREDIT_SUM_LIMIT_sum,bd_AMT_CREDIT_SUM_LIMIT_mean,bd_AMT_CREDIT_SUM_LIMIT_max,bd_AMT_CREDIT_SUM_OVERDUE_max,bd_AMT_CREDIT_SUM_OVERDUE_mean,bd_DAYS_CREDIT_UPDATE_max,bd_DAYS_CREDIT_UPDATE_min,bd_DAYS_CREDIT_UPDATE_mean,bd_bb_31_60PD_max_sum,bd_bb_0PD_max_sum,bd_bb_120PD_MORE_max_sum
188905,319992,9,8,10,7,0,0,-1128.411765,-82,-2753,0,0.0,3425.125000,30903.0,-2387.0,0,0.00,119327.656765,430605.0,40922.470588,204678.0,33820.83,2415.773571,33820.83,0.0,0.0,-5,-1672,-566.764706,0.0,16.0,0.0
297674,446761,1,3,4,0,0,0,-1724.000000,-145,-2763,0,0.0,-1211.500000,586.0,-2582.0,1,0.25,130024.125000,211500.0,102195.000000,204390.0,0.00,0.000000,0.00,0.0,0.0,-105,-2539,-1380.250000,0.0,0.0,0.0
7927,109217,5,8,10,2,0,1,-369.538462,-6,-2758,0,0.0,2632.181818,31097.0,-1663.0,0,0.00,72536.192308,242014.5,14268.600000,128106.0,0.00,0.000000,0.00,0.0,0.0,-1,-723,-118.692308,0.0,0.0,0.0


# JOIN FINAL


In [ ]:
df_prev=app_data.merge(agg_prev_app, on='SK_ID_CURR',how='left')
df_prev=df_prev.merge(agg_pos_cash_data,on='SK_ID_CURR',how='left')
df_prev=df_prev.merge(agg_cc_data,on='SK_ID_CURR',how='left')
df_prev=df_prev.merge(agg_ip_data,on='SK_ID_CURR',how='left')
df_prev=df_prev.merge(agg_bureau_data,on='SK_ID_CURR',how='left')

In [ ]:
df_prev.shape

(307511, 200)

In [ ]:
df_prev.to_pickle("df.pkl")